<a href="https://colab.research.google.com/github/Isafon/CS356Labs25/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: RSA Encryption

RSA is an asymmetric encryption scheme that follows three principle steps.
* Key Generation
* Encryption using Public Key
* Decryption using Private Key



### Key Generation

Key generation is somewhat complicated as unlike symmetric encryption schemes you are generating a key pair, or two keys that are mathematically linked together.

The steps are:
* Choosing prime numbers $p$ and $q$
* Find $n$ the product of the primes $n = p*q$
* Calculate the Euler Totient function $Φ(n) = (p - 1) * (q - 1)$

The Euler Totient calculates the number of numbers up to n that are relatively prime or coprime to n; meaning that they share no common factors with n other than 1.

* Find the encryption exponent $e$ where $1 < e < Φ(n)$ and $gcd(e, Φ(n)) = 1$

In other words, e must be coprime to the Euler Totient function $Φ(n)$, this allows for the creation of $d$.

* Calculate decryption exponent  $d$  where $d * e = 1 \mod Φ(n) $

d has a modular multiplicative inverse relationship with e, and there may be multiple keys that fit that relationship, all of them are valid.

The public and private keys are then built using those calulated numbers paired together.

public key = ($n , e$)
private key = ($n , d$)


In [ ]:
#functions needed for calculations
#calculate the modular multiplicative inverse, python 3.8+ provides a built in use of the pow function to accomplish this d = pow(e, -1, n) where e is the encryption exponent and n is the Euler Totient
import math as Math
def modInverse(e, n):
    return pow(e, -1, n)




#### Choosing Prime Numbers
##### Why prime numbers?
Prime numbers are unique in that they cannot be factored any further, for instance, the number 13 is the smallest it can be evenly reduced through division. This property is especially useful for generating keys, as when trying to guess a key if non-prime numbers are used to generate it, the factors of that generator number may be able to be used to deduce the non prime number. As RSA's encryption strength is based around the difficulty factoring very large numbers, adding the possibility of smaller factors through non prime numbers decreases the encryption strength.



Choosing prime numbers is a bit arbitrary however the strength of your encryption is directly related the size of your prime number.The reason for this is the product of primes n which is both part of the keys and is directly related to the prime numbers $p$ and $q$. if $p$ and $q$ are small prime numbers it is relatively easy to find the factor pairs that multiply to find n from n. Take for instance

$p = 31,
q = 23,
n = 713$

Even by hand, finding the factors of 713 is not an exceedingly difficult task as it has relatively few factors and a smaller search space for those factors.

Factors of 713: $1, 31, 23, 713$

Compare that to trying to find the factors of 684,623. This is not as easy of a task to human mathematicians, though not impossible but still more simple to computers.

1024 or 2048 bit prime number prodcuts are much harder to factor, the general rule is that the strength of the key depends on the size of the ***smallest*** prime number used to create $n$ because if either number is found both are found $q = n/p$ and $p = n/q$ and $n$ is a known part of the public key.

The general rule is that if an attacker can find out what $p$ and $q$ are, they can easily search the possible keys generated from $p$ and $q$ for the decryption key/ private key.

For this exercise, in the interest of execution time we will use somewhat small prime numbers.

$p = 7793$
$q = 3659$



In [ ]:

def genKey(p, q):
  #p, q = 94781 , 82153
  #Calculate n
  n = p*q
  #Find the Euler Totient
  Etot = (p-1)*(q-1)
  e = 0
  #Calculate a e that is coprime to the totient, it shares no factors other than 1
  for e in range(2, Etot):
      if Math.gcd(e, Etot) == 1:
            break
  #Find a d that has a modular inverse relationship with e and the totient
  d = modInverse(e, Etot)

  return (e, d, n)
e, d, n = genKey(7793,3659)
print(f'e is {e}, d is {d}, n is {n}')



#### Encryption

Encryption in RSA is relatively simple, however, as it is computation based it can only be done on numbers, as such, any text must be converted or "encoded" into a numerical format. One way that this is done is by converting a string into its ascii values and appending them to create a long number. That number is then encrypted using RSA.

The general form is
$M^e\mod n$
Where M is the message, e is the encryption exponent and n is the product of primes.
This is able to work due to the coprime relationship with the totient that is maintained between e an d which allows for decryption.

In [3]:
def encrypt(e, n, M):
  #Encryption takes the message raises it to the power of e (the encryption exponent) then takes the modulo of that product by n
  return M**e % n

orgtext = int(input('Please input a number Message: '))
cipher = encrypt(e, n, orgtext)
print(f'Cipher text is {cipher}')

Please input a number Message: 888883838383
Cipher text is 27217344


#### Decryption
Like encryption, decryption is relatively simple, in fact it follows the same form as encryption but with different keys. in fact, many implementations just create a single function to perform modular exponentiation, the operation required.

The form for decryption is $C^d\mod n$ where C is the cipher text, d is the encryption exponent and n is the product of primes.
Its output is the numerical encoded form of the message that was encrypted earlier.


In [ ]:
def decrypt(d, n, C):
  #Decryption takes the cipher text, raises it to the power of d (the decryption exponent) and then takes the modulo of that product by n
  return C**d % n
plain = decrypt(d, n , cipher)
print(f'Decrypted text is {plain}')


#### Does which key is which matter?

The public and private keys can each seperate encrypt or decrypt as the same mathematical operations are performed and they share the inverse modular relationship. However, as there is no specific requirement that $e$ is chosen at random from the possible comprimes of the totient, it is often chosen to be a smaller or more convenient number. You'll see the algorithm we provided makes no consideration for a random $e$, as such nonrandom $e$ values may be easier for an attacker to guess or brute force. Using that value to decrypt instead of encrypt may enable attackers to decrypt the message by guessing the decryption exponent.

In [ ]:
#this shows that you can encrypt with d and decrypt with e.
x = 12
encr = decrypt(d,n,x)
print(f'encrypted is {encr}')
decr = encrypt(e,n,encr)
print(f'decrypted is {decr} ')

#### Assignment

Your job is to create experiments with different sizes of prime numbers and lengths of messages in this notebook and record your observations. In particular you should be able to answer the following questions:
<ol>
<li> How do longer prime numbers and longer messages effect speed of the RSA process?</li>
<li> What part of the RSA process takes the most time, key generation, encryption or decryption. Why is that the slowest process?</li>
<li> Is encryption or decryption faster? Why is one faster than the other?</li>
</ol>

#### What to turn in:
A zip file containing:
* A pdf file detailing your experiments and observations, your observations should include answers to the above questions as well as any additional insights.
* This notebook including the experiments you performed and any changes to the code you made.